# Setup

Create 4 datasets-
1. dataset-1
2. dataset-2
3. dataset-3
4. dataset-4

Create 4 users
1. user-ds-1
2. user-ds-2
3. user-ds-3

The grants are as follows:
Create four datasets as follows:

| Name  | Object | Contents | Read Access| Write(And Read) Access|
|----------|----------|----------|-----------|-----------|
|  dataset-1   |  /a/b/c/ds.txt   |  `I am ds1`   |  user-ds-1/2/3   |  integration-test   |     
|  dataset-2   |  /a/b/c/ds.txt   |  `I am ds2`   |  user-ds-2   |  integration-test  |
|  dataset-3   |  /a/b/c/ds.txt   |  `I am ds3`   |  user-ds-3   |  integration-test   |
|  dataset-4   |  /a/b/c/ds.txt   |  `I am ds4`   |  integration-test    |  integration-test   |

Add more folders as needed to test listing. In `dataset-1` I created a folder structure like this

`dataset-1` 
  - `a` (Folder)
      - `b` (Folder)
          - `c` (Folder)
              - `alpha` (Folder)
              - `f1.txt` (File)
              - `f2.txt` (File)
              - `ds.txt` (File)
          - `z` (Folder)
      - `y` (Folder)
  - `x` (Folder)


In [ ]:
!pip install PyJWT

In [ ]:
import os
import requests
import json

In [ ]:
TOKEN_SVC_USER_DS_1=os.environ['TOKEN_SVC_USER_DS_1']
TOKEN_SVC_USER_DS_2=os.environ['TOKEN_SVC_USER_DS_2']
TOKEN_SVC_USER_DS_3=os.environ['TOKEN_SVC_USER_DS_3']
TOKEN_SVC_TEST_APP_1=os.environ['TOKEN_SVC_TEST_APP_1']
TOKEN_SVC_PROD_APP_1=os.environ['TOKEN_SVC_PROD_APP_1']

In [ ]:
import jwt
def get_decoded_jwt(token):
    return jwt.decode(token, options={"verify_signature": False})
    

## Get a list of Datasets inside an App

The steps to follow-
1. Get the Service Account Token
2. Make a call to the Domino API to get all datasets

In [ ]:
def get_all_datasets(token):    
    my_headers = {"Authorization": f"Bearer {token}"}
    domino_api_host = os.environ['DOMINO_API_HOST']
    endpoint = '/dataset/all'
    service_name="secure-datasets-svc"
    dns_name = f"{service_name}.domino-compute.svc.cluster.local"
    r = requests.get(f"https://{dns_name}/{endpoint}",verify=False)
    #url = f"{domino_api_host}/v4/datasetrw/datasets-v2"
    #response = requests.get(url,headers=my_headers)
    print(r.text)
    if r.status_code==200:        
        datasets = r.json()
        json_formatted_str = json.dumps(datasets, indent=2)
        print(json_formatted_str)
        



In [ ]:

def get_all_datasets(token):
    my_headers = {"Authorization": f"Bearer {token}"}
    endpoint = '/dataset/all'
    service_name="secure-datasets-svc"
    dns_name = f"{service_name}.domino-compute.svc.cluster.local"
    r = requests.get(f"https://{dns_name}/{endpoint}",headers=my_headers, verify=False)
    ret=[]
    for d in r.json()['all_datasets']:
        id = d['datasetRwDto']['id']
        name = d['datasetRwDto']['name']
        project_id = d['datasetRwDto']['projectId'] 
        ret.append({'id':id,'name':name, 'projectId':project_id})
    
    return ret

In [ ]:
print('TOKEN_INTEGRATION_TEST')
API_PROXY=os.environ['DOMINO_API_PROXY']
token = requests.get(f"{API_PROXY}/access-token").text
print(get_decoded_jwt(token)['preferred_username'])
print('TOKEN_SVC_USER_DS_1')
print(get_decoded_jwt(os.environ['TOKEN_SVC_USER_DS_1'])['preferred_username'])
print('TOKEN_SVC_USER_DS_2')
print(get_decoded_jwt(os.environ['TOKEN_SVC_USER_DS_1'])['preferred_username'])
print('TOKEN_SVC_USER_DS_3')
print(get_decoded_jwt(os.environ['TOKEN_SVC_USER_DS_1'])['preferred_username'])
print('TOKEN_SVC_TEST_APP_1')
print(get_decoded_jwt(os.environ['TOKEN_SVC_TEST_APP_1'])['preferred_username'])
print('TOKEN_SVC_PROD_APP_1')
print(get_decoded_jwt(os.environ['TOKEN_SVC_PROD_APP_1'])['preferred_username'])

In [ ]:

uri = os.environ['DOMINO_API_PROXY']
token = requests.get(f"{uri}/access-token").text
integration_test_datasets = get_all_datasets(token)
svc_user_ds_1_datasets = get_all_datasets(os.environ['TOKEN_SVC_USER_DS_1'])
svc_user_ds_2_datasets = get_all_datasets(os.environ['TOKEN_SVC_USER_DS_2'])
svc_user_ds_3_datasets = get_all_datasets(os.environ['TOKEN_SVC_USER_DS_3'])
svc_test_app_1_datasets = get_all_datasets(os.environ['TOKEN_SVC_TEST_APP_1'])
svc_prod_app_1_datasets = get_all_datasets(os.environ['TOKEN_SVC_PROD_APP_1'])

In [ ]:
print('integration_test')
print(json.dumps(integration_test_datasets, indent=2))
print('svc_user_ds_1')
print(json.dumps(svc_user_ds_1_datasets, indent=2))
print('svc_user_ds_2')
print(json.dumps(svc_user_ds_2_datasets, indent=2))
print('svc_user_ds_3')
print(json.dumps(svc_user_ds_3_datasets, indent=2))
print('svc_test_app_1')
print(json.dumps(svc_test_app_1_datasets, indent=2))
print('svc_prod_app_1')
print(json.dumps(svc_prod_app_1_datasets, indent=2))


## In Development Model

In Dev Mode, a user is in their workspace. In a workspace the datasets available to the user can be read directly by going to the mount `/domino/datasets/local/{DS_NAME}`

However, if you are doing App Development we recommend that you instead you the `secure-datasets-svc` to access the dataset because-
1. Dataset mounts will not be available inside Apps
2. You want to be able to run the same code in Apps that you use in Development

In an App the dataset file is requested via the service. It takes two parameters
1. Dataset_id
2. Calling User Name (The user calling the app). We refer to this as the `APP_CALLER`

The authentication token passed to the service though is the token of the owner of the app obtained
by calling the endpoint `$DOMINO_API_PROXY/access-token`. We shall refer to this as the `APP_OWNER_TOKEN` and the owner of this token as the `APP_OWNER`

The `secure-datasets-svc` will return the dataset file to the caller if the following two conditions are satified
1. The `APP_OWNER` has permissions to read the dataset
2. The `APP_CALLER` has permissions to read the dataset

>> The important thing to note is the APP can put any username as the `CALLING_USER_NAME`. However,
>> unless the APP_OWNER has permissions to read the dataset, the call will return a 403 error.
>> The access to the dataset is bounded by the permissions of the APP_OWNER. This is a strong
>> constraint. The proper process is to move the right files into a production datasets. Give
>> the APP_OWNER access to these datasets. And then select the right APP_CALLER's the permissions
>> to read the appropriate subset of these production datasets. This allows the enforcement of data
>> access control to be a process defined by the owner of the data. **This will get clearer as
>> we go along**
   

In [ ]:
## Development Mode - Access Dataset

## We can verify access to the datasets
def get_app_owner_token():
    uri = os.environ['DOMINO_API_PROXY']
    app_owner_token = requests.get(f"{uri}/access-token").text
    return app_owner_token
def verify_dataset_access(token,dataset_id,app_caller):
    my_headers = {"Authorization": f"Bearer {token}"}
    #The nucleus always adds this header when the call to the app is made
    #There is ongoing work to replace this with a JWT to make it impossible 
    #for Apps to insert a wrong value here when calling the secure datasets
    #service
    my_headers['domino-username']=app_caller
    endpoint = f'/dataset/canread/{dataset_id}'
    service_name="secure-datasets-svc"
    dns_name = f"{service_name}.domino-compute.svc.cluster.local"
    r = requests.get(f"https://{dns_name}/{endpoint}",headers=my_headers, verify=False)
    return r.json()


We are currently operating as the user `integration_test` which is an admin with access to all datasets. In practice your development user will not have access to all datasets. But that is not relevant. The code below will always work because you are emulating an `APP_OWNER`. You first get access to only the datasets the `APP_OWNER` sees


In [ ]:
integration_test_datasets = get_all_datasets(get_app_owner_token())
'''
for d in integration_test_datasets:
    print(d)
'''

Now for each of these datasets you can verify if the following users have access to or not. These are all named users in Domino. Service Accounts are not included

| User               | Role          | Is Power User for Secure Datasets Service | 
|--------------------|---------------|-------------------------------------------|
| `integration-test` | Administrator | Yes                                       |
| `user-ds-1`        | Practitioner  | No                                        |
| `user-ds-2`        | Practitioner  | No                                        |
| `user-ds-3`        | Practitioner  | No                                        |
| `dev-app-1-owner`  | Practitioner  | No                                        |
| `test-app-1-owner` | Practitioner  | No                                        |
| `prod-app-1-owner` | Practitioner  | No                                        |

**We will see later that service accounts are used to start Apps. And these
define the boundary of which datasets can see Apps. But only a named Domino
user can call Apps**

In [ ]:
app_callers = ['user-ds-1',
               'user-ds-2','user-ds-3',
               'dev-app-1-owner','test-app-1-owner',
               'prod-app-1-owner']
for d in integration_test_datasets:
    id = d['id']
    name = d['name']
    for caller in app_callers:
        out = verify_dataset_access(get_app_owner_token(),d['id'],caller)
        print(f'Caller {caller} access to dataset id: {id} and name: {name}')
        print(out)

## Browse Dataset

As long as calling user has access to the datasets, we can freely browse it. If the calling user does not have access (or the app_owner does not access) we get 403 error

Let us try the following datasets:
```
  {
    "id": "662fbf1c4d68ee2895b43742",
    "name": "dataset-1",
    "projectId": "662a70a548ce7c5c14d8d3ce"
  },
  {
    "id": "662fbf244d68ee2895b43745",
    "name": "dataset-2",
    "projectId": "662a70a548ce7c5c14d8d3ce"
  },
  {
    "id": "662fbf2f4d68ee2895b43748",
    "name": "dataset-3",
    "projectId": "662a70a548ce7c5c14d8d3ce"
  }
```

In [ ]:
def list_data_set(token,dataset_id,app_caller,path):
    my_headers = {"Authorization": f"Bearer {token}"}
    #The nucleus always adds this header when the call to the app is made
    #There is ongoing work to replace this with a JWT to make it impossible 
    #for Apps to insert a wrong value here when calling the secure datasets
    #service
    my_headers['domino-username']=app_caller
    params = {'path':path}
    endpoint = f'/dataset/list/{dataset_id}'
    service_name="secure-datasets-svc"
    dns_name = f"{service_name}.domino-compute.svc.cluster.local"
    r = requests.get(f"https://{dns_name}/{endpoint}",headers=my_headers,
                     params=params, verify=False)
    return r.json()

In [ ]:
#Users

user1 = "user-ds-1"
user2 = "user-ds-2"
user3 = "user-ds-3"

# Datasets
dataset_1_id="662fbf1c4d68ee2895b43742"
dataset_2_id="662fbf244d68ee2895b43745"
dataset_3_id="662fbf2f4d68ee2895b43748"
user = user1
out = list_data_set(get_app_owner_token(),dataset_1_id,user,"")
print(json.dumps(out, indent=2))
out = list_data_set(get_app_owner_token(),dataset_1_id,user,"a")
print(json.dumps(out, indent=2))
out = list_data_set(get_app_owner_token(),dataset_1_id,user,"a/b")
print(json.dumps(out, indent=2))
out = list_data_set(get_app_owner_token(),dataset_1_id,user,"a/b/c")
print(json.dumps(out, indent=2))

## Publishing an App using an API

Using a Admission Validating Webhook, we have disabled a user access to an App
Apps are only permitted to be published via a Service Accounts. 

And more so each of these service account has been designated as one of the three types

1. dev (only for workspaces) 
2. dev-test (when a service account is assigned to a named domino user for dev testing)
3. test (when a service account is assigned to a user responsible for testing the app)
4. prod (when a service account is assigned to a user responsible for deploying app to production)

You can have many more types of environments. But `dev-test` is always used for development. The rest can be custom defined.

If you want to find out which service account is assigned to which env look for the file
`/etc/config/sa-to-env-mapping.properties` . For this project and user `integration-test` we have mounted it. In general it will not be mounted for any user. You can designate a user and domino project id where each of these will be mounted

File = `/etc/config/sa-to-env-mapping.properties`
Folders = `/dev-test/`,  `/dev/`, `/test/` and `/prod/`

Example contents of this file are-

```
svc-user-ds-1=dev-test
svc-user-ds-2=dev-test
svc-user-ds-3=dev-test
svc-test-app-1=test
svc-prod-app-1=prod
```

Inside a workspace for the `dev` subfolder only the following will be mounted specifically

`/dev/secure/datasets/data/{user-name}`
`/dev/secure/datasets/metadata/{user-name}`

We will see why soon. In general a designated user and project id will be allowed to debug from workspaces so the workspace for this project and user see exactly what the `secure-datasets-svc` sees



## Starting an App using an API

Users cannot start an App from the UI. Only designated service accounts which are part of the file 
`/etc/config/sa-to-env-mapping.properties` can start apps

Starting an App first time using an API is done by the following Domino API call
```
curl -X POST "https://prod-field.cs.domino.tech/v4/modelProducts" -H  "accept: application/json" -H  "Content-Type: application/json" -d "{\"modelProductType\":\"APP\",\"projectId\":\"string\",\"name\":\"string\",\"description\":\"string\",\"publisher\":{\"lastName\":\"string\",\"firstName\":\"string\",\"phoneNumber\":\"string\",\"avatarUrl\":\"string\",\"idpId\":\"string\",\"companyName\":\"string\",\"fullName\":\"string\",\"id\":\"string\",\"userName\":\"string\",\"email\":\"string\"},\"created\":\"2024-06-13T15:42:04.075Z\",\"lastUpdated\":\"2024-06-13T15:42:04.075Z\",\"status\":\"string\",\"media\":[\"string\"],\"openUrl\":\"string\",\"projectUrl\":\"string\",\"tags\":[\"string\"],\"stats\":{\"usageCount\":0},\"appExtension\":{\"appType\":\"string\"},\"id\":\"string\",\"latestAppVersionId\":\"string\",\"runningAppUrl\":\"string\",\"runningCommitId\":\"string\",\"environmentId\":\"string\",\"hardwareTierId\":\"string\",\"permissionsData\":{\"appAccessStatus\":\"ALLOWED\",\"pendingInvitations\":[\"string\"],\"visibility\":\"PUBLIC\",\"discoverable\":true,\"accessRequestStatuses\":{\"additionalProp1\":\"string\",\"additionalProp2\":\"string\",\"additionalProp3\":\"string\"}},\"runDescribeUrl\":\"string\",\"goalIds\":[\"string\"]}"
```

Note the modelId. Or you can simply start as a user and note the modelId from the URL. The app would fail but you would get the modelId

## More details of the App

Let us start with the file `/etc/config/sa-to-env-mapping.properties`

Example contents of this file are-

```
svc-user-ds-1=dev-test
svc-user-ds-2=dev-test
svc-user-ds-3=dev-test
svc-test-app-1=test
svc-prod-app-1=prod
```
You can only start an App with these 5 svc accounts. Users `user-ds-[1,2,3]` are respectively assigned `svc-user-ds-[1,2,3]`

Now pivoting to the `secure-datasets-svc`. It has the following folders mounted-
```
/secure/datasets/data
/secure/datasets/metadata
```

Each time this service receives a request to retrieve a file in `dataset-id` and `path-to-a-file` in the dataset, it will place the file with a randomly generated file name in exactly one and only one set of these sets of folder.

The request is made with a time-to-live for this file (default is 300 seconds). The actual file is placed in the folder

```
/secure/datasets/data/{app-owner}/{calling-user}/{random-file-name}
/secure/datasets/metadata/{app-owner}/{calling-user}/{random-file-name}
```

The file in the `metadata` determines how long the file will be avaiable. These folders are tracked once every minute and expired files get deleted. 

Now back to the App and Workspace. 

First the workspace. Inside each workspace the following folders are mounted
```
/dev/secure/datasets/data/{domino-starting-user}
/dev/secure/datasets/metadata/{domino-starting-user}
```

When the `secure-datasets-svc` gets a call to retrieve a file and the call is not from an `APP_OWNER` mentioned in the `/etc/config/sa-to-env-mapping.properties` it adds the file to the folders
```
/dev/secure/datasets/data/{app-caller}
/dev/secure/datasets/metadata/{app-caller}
```
Note that inside a workspace the precise `domino-starting-user` sub-folder is mounted. So it makes little sense to call this from a workspace with a `app-calling-user` not equal to the `domino-starting-user`. This mode is to allow 

In [ ]:
def fetch_data_set(token,dataset_id,app_caller,path):
    my_headers = {"Authorization": f"Bearer {token}"}
    #The nucleus always adds this header when the call to the app is made
    #There is ongoing work to replace this with a JWT to make it impossible 
    #for Apps to insert a wrong value here when calling the secure datasets
    #service
    my_headers['domino-username']=app_caller
    params = {'path':path}
    endpoint = f'/dataset/fetch/{dataset_id}'
    service_name="secure-datasets-svc"
    dns_name = f"{service_name}.domino-compute.svc.cluster.local"
    r = requests.get(f"https://{dns_name}/{endpoint}",headers=my_headers,
                     params=params, verify=False)
    return r.json()

In [ ]:
user = os.environ['DOMINO_USER_NAME']
# Datasets
dataset_1_id="662fbf1c4d68ee2895b43742"
out = verify_dataset_access(get_app_owner_token(),dataset_1_id,user)
print('Is Dataset accessible to the caller')
print(json.dumps(out, indent=2))

out = fetch_data_set(get_app_owner_token(),dataset_1_id,user,"a/b/c/ds.txt")
print(out)
print('Fetching dataset file')
print(json.dumps(out, indent=2))


# What just happened

The file `a/b/c/ds.txt` was returned in the location `/dev/secure/datasets/data/integration-test/1718299135-501024`. Let us confirm they are the same

In [ ]:
!cat /domino/datasets/local/dataset-1/a/b/c/ds.txt

In [ ]:
original_file='/domino/datasets/local/dataset-1/a/b/c/ds.txt'
print(f'Contents of file {original_file}')
with open(out['local_path'], 'r') as file:
    # Read the entire content of the file
    content = file.read()
    print(content)

copied_file=out['local_path']
print(f'Contents of file {copied_file}')
# Open the file in read mode ('r')
with open(copied_file, 'r') as file:
    # Read the entire content of the file
    content = file.read()
    print(content)


## Now we switch to service accounts to start an App

User `user-ds-[1,2,3]` are assigned svc accounts `svc-user-ds-[1,2,3]` respectively. Starting and stopping Apps based on service accounts is documented at the end of this notebook. For now let us pretend we are inside the app and try with each of those three service account

In [ ]:
svc_user_ds_1_datasets = get_all_datasets(os.environ['TOKEN_SVC_USER_DS_1'])
svc_user_ds_2_datasets = get_all_datasets(os.environ['TOKEN_SVC_USER_DS_2'])
svc_user_ds_3_datasets = get_all_datasets(os.environ['TOKEN_SVC_USER_DS_3'])
svc_test_app_1_datasets = get_all_datasets(os.environ['TOKEN_SVC_TEST_APP_1'])
svc_prod_app_1_datasets = get_all_datasets(os.environ['TOKEN_SVC_PROD_APP_1'])




print(json.dumps(svc_user_ds_1_datasets, indent=2))
print(json.dumps(svc_test_app_1_datasets, indent=2))
print(json.dumps(svc_prod_app_1_datasets, indent=2))

In [ ]:
dataset_id="66635ef6b8d4382572ef2aa7"
out = verify_dataset_access(os.environ['TOKEN_SVC_TEST_APP_1'],dataset_id,'user-ds-3')
print('Is Dataset accessible to the caller')
print(json.dumps(out, indent=2))

out = fetch_data_set(os.environ['TOKEN_SVC_TEST_APP_1'],dataset_id,user,"a/b/c/ds.txt")
print(out)
print('Fetching dataset file')
print(json.dumps(out, indent=2))
original_file='/domino/datasets/local/test-dataset-1/a/b/c/ds.txt'
print(f'Contents of file {original_file}')
with open(out['local_path'], 'r') as file:
    # Read the entire content of the file
    content = file.read()
    print(content)

copied_file=out['local_path']
print(f'Contents of file {copied_file}')
# Open the file in read mode ('r')
with open(copied_file, 'r') as file:
    # Read the entire content of the file
    content = file.read()
    print(content)


In [ ]:
dataset_id="66635ef6b8d4382572ef2ab0"
out = verify_dataset_access(os.environ['TOKEN_SVC_PROD_APP_1'],dataset_id,'user-ds-3')
print('Is Dataset accessible to the caller')
print(json.dumps(out, indent=2))

out = fetch_data_set(os.environ['TOKEN_SVC_PROD_APP_1'],dataset_id,user,"a/b/c/ds.txt")
print(out)
print('Fetching dataset file')
print(json.dumps(out, indent=2))
original_file='/domino/datasets/local/prod-dataset-1/a/b/c/ds.txt'
print(f'Contents of file {original_file}')
with open(out['local_path'], 'r') as file:
    # Read the entire content of the file
    content = file.read()
    print(content)

copied_file=out['local_path']
print(f'Contents of file {copied_file}')
# Open the file in read mode ('r')
with open(copied_file, 'r') as file:
    # Read the entire content of the file
    content = file.read()
    print(content)


## Stopping the App

Execute the following endpoint to stop an App

```
import os
import requests
domino_api_host = os.environ['DOMINO_API_HOST']
app_id=<ADD YOUR APP ID HERE>
url = f"{domino_api_host}/v4/modelProducts/{app_id}/stop?force=true"

# Fetch the mounted service account token
token_file = "/etc/tokens/token"
# Open the file in read mode
with open(token_file, 'r') as file:
    # Read the entire file content into a variable
    sa_token = file.read()

# Now, file_content variable holds the content of the file
print(sa_token)
headers = {
  'Content-Type': 'application/json',
  'Authorization': f"Bearer {sa_token}"
  }
response = requests.request("POST", url, headers=headers, data={})

print(response.status_code)
print(response.text)
```


In [ ]:
#stop
import os
import requests
domino_api_host = os.environ['DOMINO_API_HOST']
app_id="6632694cb0361c7a46730640"
sa_token = os.environ['TOKEN_SVC_USER_DS_1']
url = f"{domino_api_host}/v4/modelProducts/{app_id}/stop?force=true"


headers = {
  'Content-Type': 'application/json',
  'Authorization': f"Bearer {sa_token}"
  }
response = requests.request("POST", url, headers=headers, data={})

if response.status_code==200:
    print(f"App with id {response.text} stopped")
else:
    print(f"Error - Response Code {response.status_code}, message = {response.text} ")


## Starting an App
```
import os
import requests
import json

domino_api_host = os.environ['DOMINO_API_HOST']
app_id="6632694cb0361c7a46730640"
url = f"{domino_api_host}/v4/modelProducts/{app_id}/start"


# Fetch the mounted service account token
token_file = "/etc/tokens/token"
# Open the file in read mode
with open(token_file, 'r') as file:
    # Read the entire file content into a variable
    sa_token = file.read()


payload = json.dumps({
  "hardwareTierId": "small-k8s",
  "environmentId": "662a613292c1117e2b322d02",
  "externalVolumeMountIds": []
})
headers = {
  'Content-Type': 'application/json',
  'Authorization': f"Bearer {sa_token}"
  }

response = requests.request("POST", url, headers=headers, data=payload)
print(response.status_code)
print(response.text)
```

In [ ]:
import os
import requests
import json

domino_api_host = os.environ['DOMINO_API_HOST']
app_id="6632694cb0361c7a46730640"

#app_id="663a3b3f1aad977129add91c"
url = f"{domino_api_host}/v4/modelProducts/{app_id}/start"

payload = json.dumps({
  "hardwareTierId": "small-k8s",
  "environmentId": "662a613292c1117e2b322d02",
  "externalVolumeMountIds": []
})
d={
        "name": "token-for-svc-prod-app-1",
        "serviceAccountIdpId": "928ef5af-f51e-4832-a213-4ba432b0beeb",
        "token": "xxxxx",
        "isValid": True,
        "createdAt": "2024-06-07T17:06:27.099Z",
        "expiresAt": "2024-10-05T17:06:27.092096291Z"
    }

headers = {
  'Content-Type': 'application/json',
  'Authorization': f"Bearer {sa_token}"
  }

response = requests.request("POST", url, headers=headers, data=payload)
if response.status_code==200:
    print(f"App with id {response.text} started")
else:
    print(f"Error - Response Code {response.status_code}, message = {response.text} ")
          


In [ ]:
url = os.environ['DOMINO_API_HOST'] + "/v4/modelProducts/6632694cb0361c7a46730640"
# Fetch the mounted service account token
token_file = "/etc/tokens/token"
# Open the file in read mode

headers = {
  'Content-Type': 'application/json',
  'Authorization': f"Bearer {sa_token}"
  }
response = requests.request("GET", url, headers=headers, data={})
response.json()

In [ ]:
get_decoded_jwt(get_app_owner_token())